**Update these values, and then Restart and Run All**

In [1]:
Q = 3
YEAR = 2022

In [2]:
# Parameters
Q = 3
YEAR = 2022


In [3]:
import pandas as pd
import genetic_collections as gc
import tarfile
import io
from pathlib import Path

## Downloading GenBank Records

In [4]:
q_end_dates = {1: '3/31',
               2: '6/30',
               3: '9/31',
               4: '12/31'}
bioprojects = {'GGI': '384793',
               'SIBN': '81359'}

In [5]:
project_results = []
for initiative, bioproject_code in bioprojects.items():
    gb_query = f'{bioproject_code}[BioProject] AND ("1900/01/01"[PDAT] : "{YEAR}/{q_end_dates[Q]}" [PDAT])'
    gb_search_results = gc.gb_search(raw_query=gb_query)
    print(initiative, len(gb_search_results.id_list))
    gb_results = gc.gb_fetch_from_id_list(gb_search_results.id_list)
    print(initiative, len(gb_results))
    project_df = pd.DataFrame(gb_results)
    project_df['initiative'] = initiative
    print(project_df.info())
    project_results.append(project_df)

GGI 14251
GGI 14251
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   accession         14251 non-null  object
 1   scientific_name   14251 non-null  object
 2   publish_date      14251 non-null  object
 3   update_date       14251 non-null  object
 4   keyword           14251 non-null  object
 5   bioproject        14251 non-null  object
 6   seq_len           14251 non-null  object
 7   submit_authors    14251 non-null  object
 8   submit_date       14251 non-null  object
 9   submit_inst       14251 non-null  object
 10  specimen_voucher  14201 non-null  object
 11  taxid             14251 non-null  object
 12  country           14196 non-null  object
 13  collection_date   12313 non-null  object
 14  collected_by      12213 non-null  object
 15  PCR_primers       12106 non-null  object
 16  lat_lon           11668 non-null  obje

In [6]:
gb_df = pd.concat(project_results)
gb_df['quarter'] = pd.to_datetime(gb_df['publish_date']).dt.year.astype(str) + '-Q' +\
                     pd.to_datetime(gb_df['publish_date']).dt.quarter.astype(str)
keep_cols = ['initiative','bioproject','accession','specimen_voucher','publish_date','quarter','taxid']
gb_df = gb_df[keep_cols]
gb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46826 entries, 0 to 32574
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   initiative        46826 non-null  object
 1   bioproject        46826 non-null  object
 2   accession         46826 non-null  object
 3   specimen_voucher  46754 non-null  object
 4   publish_date      46826 non-null  object
 5   quarter           46826 non-null  object
 6   taxid             46826 non-null  object
dtypes: object(7)
memory usage: 2.9+ MB


In [7]:
gb_df.head()

,initiative,bioproject,accession,specimen_voucher,publish_date,quarter,taxid
0,GGI,PRJNA801314,OM460701,US:D&ML 70468,30-MAR-2022,2022-Q1,74090
1,GGI,PRJNA801314,OM460700,US:D&ML 69713,30-MAR-2022,2022-Q1,2071691
2,GGI,PRJNA801314,OM460699,US:D&ML 68816,30-MAR-2022,2022-Q1,2877961
3,GGI,PRJNA801314,OM460698,US:D&ML 68293,30-MAR-2022,2022-Q1,2306629
4,GGI,PRJNA801314,OM460697,US:D&ML 63797,30-MAR-2022,2022-Q1,2116083


In [8]:
gb_df['taxid'] = gb_df['taxid'].astype(int)

## Downloading NCBI Taxonomy

In [11]:
#hydra_tax_dir = Path('/scratch/dbs/blast/taxonomy')
#new_tax_loc = hydra_tax_dir / 'new_taxdump.latest.tar.gz'
new_tax_loc = 'new_taxdump.tar.gz'

with tarfile.open(new_tax_loc, "r:gz") as tar:
    rankedlineage = tar.extractfile('rankedlineage.dmp').read()
    ranked_taxa = pd.read_csv(io.BytesIO(rankedlineage), 
                     encoding='utf8', 
                     sep='\t\|\t', engine='python',
                     error_bad_lines=False,
                     names=['taxid','tax_name','species','genus','family',
                            'order','class','phylum','kingdom','superkingdom'])
ranked_taxa['superkingdom'] = ranked_taxa['superkingdom'].str.rstrip('\t|')
ranked_taxa.head()

,taxid,tax_name,species,genus,family,order,class,phylum,kingdom,superkingdom
0,1,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,131567,cellular organisms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,2157,Archaea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,1935183,Asgard group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea
4,2798909,Candidatus Baldrarchaeota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Archaea


## Combining GenBank with Taxonomy and exporting report

In [12]:
unique_taxids = gb_df['taxid'].unique()
print(len(unique_taxids))
print(unique_taxids[:10])

20332
[  74090 2071691 2877961 2306629 2116083 2850695  911171  437227 2575620
  485435]


In [13]:
taxa_subset = ranked_taxa[ranked_taxa['taxid'].isin(unique_taxids)]
taxa_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20332 entries, 92895 to 2209809
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   taxid         20332 non-null  int64 
 1   tax_name      20332 non-null  object
 2   species       346 non-null    object
 3   genus         17577 non-null  object
 4   family        19625 non-null  object
 5   order         19936 non-null  object
 6   class         20040 non-null  object
 7   phylum        20323 non-null  object
 8   kingdom       20210 non-null  object
 9   superkingdom  20332 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.7+ MB


In [14]:
merged_df = gb_df.merge(taxa_subset[['taxid','family','genus']], on='taxid')
print(merged_df.head())

  initiative   bioproject accession specimen_voucher publish_date  quarter  \
0        GGI  PRJNA801314  OM460701    US:D&ML 70468  30-MAR-2022  2022-Q1   
1        GGI  PRJNA801314  OM460700    US:D&ML 69713  30-MAR-2022  2022-Q1   
2        GGI  PRJNA801314  OM460699    US:D&ML 68816  30-MAR-2022  2022-Q1   
3        GGI  PRJNA801314  OM460698    US:D&ML 68293  30-MAR-2022  2022-Q1   
4        GGI  PRJNA801314  OM460697    US:D&ML 63797  30-MAR-2022  2022-Q1   

     taxid            family           genus  
0    74090      Sargassaceae       Sargassum  
1  2071691    Spongitidaceae  Neogoniolithon  
2  2877961  Scytosiphonaceae   Hydroclathrus  
3  2306629     Rhodomelaceae         Amansia  
4  2116083  Peyssonneliaceae      Ramicrusta  


In [15]:
merged_outfile = f'GB_Taxonomy_Results_{YEAR}_Q{Q}.tsv'
merged_df.to_csv(merged_outfile, sep='\t', index=False)

## Calculating Quarterly Totals

In [16]:
merged_df.groupby(['initiative','quarter']).size()

initiative  quarter
GGI         2016-Q1     145
            2017-Q1       8
            2017-Q2     107
            2017-Q3    2034
            2017-Q4    3043
            2018-Q1     372
            2018-Q2    1856
            2018-Q3     165
            2018-Q4    1314
            2019-Q1       2
            2019-Q2      28
            2019-Q3     400
            2019-Q4      86
            2020-Q2      61
            2020-Q4     193
            2021-Q1     113
            2021-Q2     659
            2021-Q3     346
            2021-Q4     359
            2022-Q1    2960
SIBN        2011-Q4    2778
            2014-Q2      12
            2015-Q1     805
            2015-Q3     966
            2015-Q4    1836
            2016-Q1     534
            2016-Q2     498
            2016-Q4     256
            2017-Q2      27
            2017-Q4    1947
            2018-Q1     920
            2018-Q2     906
            2018-Q3    2266
            2018-Q4    1728
            2019-Q1     380


In [17]:
quarter_target = f'{YEAR}-Q{Q}'

sibn_q = merged_df[(merged_df['initiative'] == 'SIBN') & (merged_df['quarter'] == quarter_target)]
sibn_total = merged_df[merged_df['initiative'] == 'SIBN']
ggi_q = merged_df[(merged_df['initiative'] == 'GGI') & (merged_df['quarter'] == quarter_target)]
ggi_total = merged_df[merged_df['initiative'] == 'GGI']
both_q = merged_df[merged_df['quarter'] == quarter_target]

In [18]:
from collections import OrderedDict

In [19]:
df_labels = OrderedDict([(f'SIBN Q{Q}', sibn_q),
                         ('SIBN Total', sibn_total),
                         (f'GGI Q{Q}', ggi_q),
                         ('GGI Total', ggi_total),
                         (f'Both Initiatives Q{Q}', both_q),
                         ('Both Initiatives Total', merged_df)])

In [20]:
counts_dict = OrderedDict()
for title, df in df_labels.items():
    counts_dict[title] = OrderedDict([
                            ('Sequence Records', len(df)),
                            ('Specimens', df['specimen_voucher'].nunique()),
                            ('Families', df['family'].nunique()),
                            ('Genera', df['genus'].nunique())])
count_df = pd.DataFrame(counts_dict)
count_df.style

,SIBN Q3,SIBN Total,GGI Q3,GGI Total,Both Initiatives Q3,Both Initiatives Total
Sequence Records,605,32575,0,14251,605,46826
Specimens,596,29698,0,7397,596,37095
Families,132,1205,0,1046,132,1865
Genera,357,7551,0,3249,357,10349


In [21]:
count_df.to_csv(f'GenBank_Counts_{YEAR}_Q{Q}.tsv', sep='\t')

In [22]:
summary_file = f'GenBank_Counts_{YEAR}_Q{Q}.xlsx'
count_df.to_excel(summary_file)